In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
def preprocess_image(image):
    # Преобразуем изображение в черно-белое
    image = image.convert('L')
    
    # Преобразуем в numpy-массив
    image = np.array(image)
    
    # Нормализуем изображение
    image = image / 255.0  # нормализация в диапазоне [0, 1]
    
    # Добавляем случайные отступы
    padded_image = np.pad(image, pad_width=10, mode='constant', constant_values=0)
    
    # Возвращаем изображение в виде тензора PyTorch
    return torch.tensor(padded_image, dtype=torch.float).unsqueeze(0)  # (1, H, W)


In [3]:
class CNNEncoder(nn.Module):
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # Входное изображение
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Пуллинг для уменьшения размерности

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)  # Снижение размерности
        return x

In [11]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEmbedding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model, 16, 16)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        
    def forward(self, x):
        a = self.encoding[:x.size(0), :]
        return x + self.encoding[:x.size(0), :]

In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.W = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b = nn.Parameter(torch.randn(hidden_size))

    def forward(self, encoder_output, decoder_input):
        # Вычисляем внимание
        score = torch.matmul(decoder_input, self.W)
        score = torch.matmul(score, encoder_output.transpose(1, 2))  # Вектор внимания
        attention_weights = torch.softmax(score, dim=-1)
        context_vector = torch.matmul(attention_weights, encoder_output)  # Получаем контекстный вектор
        return context_vector, attention_weights

class DecoderLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DecoderLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, encoder_output, decoder_input):
        # Применение внимания
        context_vector, _ = self.attention(encoder_output, decoder_input)
        lstm_out, _ = self.lstm(context_vector)
        output = self.fc_out(lstm_out)
        return output

In [6]:
class MERModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MERModel, self).__init__()
        self.encoder = CNNEncoder()
        self.positional_embedding = PositionalEmbedding(d_model=256)
        self.decoder = DecoderLSTM(input_dim=256, hidden_dim=hidden_dim, output_dim=output_dim)

    def forward(self, x):
        # Шаг 1: Обработка изображения
        x = self.encoder(x)
        
        # Шаг 2: Добавление позиционных встраиваний
        x = self.positional_embedding(x)
        
        # Шаг 3: Генерация выходной последовательности
        x = self.decoder(x, x)
        
        return x

In [12]:
# Параметры модели
input_dim = 256  # Размерность карты признаков
hidden_dim = 512  # Размер скрытого состояния LSTM
output_dim = 50  # Количество символов в выходной строке

# Инициализация модели
model = MERModel(input_dim, hidden_dim, output_dim)

# Определение потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Пример данных
sample_image = torch.rand(1, 1, 128, 128)  # Случайное изображение для примера

# Обучение
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    output = model(sample_image)
    loss = criterion(output.view(-1, output_dim), torch.randint(0, output_dim, (1, 50)).view(-1))
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


RuntimeError: The expanded size of the tensor (16) must match the existing size (128) at non-singleton dimension 3.  Target sizes: [512, 128, 16, 16].  Tensor sizes: [512, 128]